# ASSIGNMENT 1 - EXCERCISE 2

### Solve a travelling salesman problem for 10 random matrices

#### The following example will be shown with full detail for a 5x5 matrix. Since the assignment requires to solve 10 different matrices, only the final result will be shown for the remaining 9 tests

In [1]:
from pulp import *

In [2]:
#Create a origin-destination matrix of the distances between the cities
#Note that the center diagonal is of 0's since the pair is origin i = destination j
#First instance will be created with 5 cities
import random
def arrange(x, rows, cols):
    random.shuffle(x)
    return [x[cols * i : cols * (i + 1)] for i in range(rows)]
origin_destination_matrix = (arrange(list(range(5*5)), 5, 5))
print(origin_destination_matrix)

[[4, 1, 7, 20, 13], [18, 17, 21, 2, 9], [12, 24, 11, 5, 22], [14, 3, 8, 10, 6], [16, 23, 15, 19, 0]]


In [3]:
#Create variables with the parameters of the matrix to work with
cities = len(origin_destination_matrix)
indexes = range(cities)
print("Cities: ", cities)
print("Indexes: ", indexes)

Cities:  5
Indexes:  range(0, 5)


In [4]:
#Create a list of all the coordinates combinations of the matrix, excluding the center diagonal of 0's
E = [(i,j) for i in indexes for j in indexes if i!=j]

In [5]:
#Instruction for minimizing the problem
m = LpProblem("TSP",LpMinimize)

In [6]:
#Create a dictionary of all coordinate pairs
#Associate each pair with an X to assign 0 or 1 afterwards if the city will be chosen as part of the optimal route
x = LpVariable.dicts('x',E,cat=LpBinary)

In [7]:
#Create the objective function of costs per each pair
#The costs are the distances stated in the origin-destination matrix, excluding the diagonal
m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])

In [8]:
#Create the constraints
#It limits to a step size of 1, where from one origin, movement can only occur to one destination at a time
#The function calculates each pair both back and forth 
for i in indexes:
    m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
    m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)

In [9]:
#Instructions for preliminar solution of the system without adding the restriction of subtours
m.solve()
vobj = value(m.objective)
xsol = {e:value(x[e]) for e in E}
print("- Optimal routes without eliminating subtours:")
sol_summary={x for x,y in xsol.items() if y!=0}
print("Segments within route: ", sol_summary)

- Optimal routes without eliminating subtours:
Segments within route:  {(0, 1), (3, 4), (2, 0), (4, 2), (1, 3)}


In [10]:
#Find successive cities that have the shortest path between them
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt

In [11]:
#Iterate the previous definition to find all possible short subtours
#The function will add more nodes as it runs out of shorter routes
while True:
    subt = subtour(xsol)
    if len(subt) == cities:
        print("- Optimal tour excluding subtours")
        print("Optimal tour found: %r"%subt)
        print("Optimal tour length: %g"%vobj)
        break
    print("- Identify subtours")
    print("Subtour found: %r"%subt)
    m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
    m.solve()
    vobj = value(m.objective)
    xsol = {e: value(x[e]) for e in E}

- Optimal tour excluding subtours
Optimal tour found: [0, 1, 3, 4, 2]
Optimal tour length: 36


#### Solution for matrices from 6x6 to 15x15

In [13]:
def arrange(x, rows, cols):
    random.shuffle(x)
    return [x[cols * i : cols * (i + 1)] for i in range(rows)]
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt
for index in range(6,16):
    print("===================================================================================================================")
    print("Matrix = ", index, "x", index)
    origin_destination_matrix = (arrange(list(range(index*index)), index, index))
    cities = len(origin_destination_matrix)
    indexes = range(cities)
    E = [(i,j) for i in indexes for j in indexes if i!=j]
    m = LpProblem("TSP",LpMinimize)
    x = LpVariable.dicts('x',E,cat=LpBinary)
    m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])
    for i in indexes:
        m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
        m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)
    m.solve()
    vobj = value(m.objective)
    xsol = {e:value(x[e]) for e in E}
    print("- Optimal routes without eliminating subtours:")
    sol_summary={x for x,y in xsol.items() if y!=0}
    print("Segments within route: ", sol_summary)

    while True:
        subt = subtour(xsol)
        if len(subt) == cities:
            print("- Optimal tour excluding subtours")
            print("Optimal tour found: %r"%subt)
            print("Optimal tour length: %g"%vobj)
            break
        print("- Identify subtours")
        print("Subtour found: %r"%subt)
        m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
        m.solve()
        vobj = value(m.objective)
        xsol = {e: value(x[e]) for e in E}

Matrix =  6 x 6
- Optimal routes without eliminating subtours:
Segments within route:  {(4, 0), (1, 2), (0, 4), (5, 1), (2, 3), (3, 5)}
- Identify subtours
Subtour found: [0, 4]
- Optimal tour excluding subtours
Optimal tour found: [0, 4, 5, 1, 2, 3]
Optimal tour length: 50
Matrix =  7 x 7
- Optimal routes without eliminating subtours:
Segments within route:  {(2, 4), (6, 2), (5, 1), (3, 0), (0, 6), (4, 5), (1, 3)}
- Optimal tour excluding subtours
Optimal tour found: [0, 6, 2, 4, 5, 1, 3]
Optimal tour length: 82
Matrix =  8 x 8
- Optimal routes without eliminating subtours:
Segments within route:  {(7, 4), (3, 1), (2, 0), (5, 7), (0, 2), (4, 5), (1, 6), (6, 3)}
- Identify subtours
Subtour found: [0, 2]
- Identify subtours
Subtour found: [0, 5, 7, 4, 2]
- Identify subtours
Subtour found: [0, 7, 4, 5, 3, 2]
- Identify subtours
Subtour found: [0, 5, 7, 4]
- Optimal tour excluding subtours
Optimal tour found: [0, 7, 4, 5, 3, 2, 1, 6]
Optimal tour length: 125
Matrix =  9 x 9
- Optimal rout